In [3]:
from blockulib.playing import ModelBasedLoop
import blockulib.models as blom
import blockulib
import torch

In [4]:
class Probe(ModelBasedLoop):
    
    def __call__(self, pos_tensor, depth = 5, batch_size = 4096, temperature = 1.0, top_k: int = None):
        num_games = pos_tensor.shape[0]
        state = [True for i in range(num_games)]
        game_length = torch.zeros(num_games)
        last_logit = torch.full((num_games, ), float('-inf'))
        active_games = num_games
        
        for d in range(depth):
            new_index = []
            for i in range(num_games):
                if state[i]:
                    new_index.append(i)
            boards = [pos_tensor[new_index[i]].clone() for i in range(active_games)]
            pos, ind = blockulib.possible_moves(boards, self.generator)
            logits = self.get_model_pred(pos, batch_size = batch_size)
            decisions = blockulib.logits_to_choices(logits, ind, active_games, temperature = temperature, top_k = top_k)
            
            #print("act ", active_games)
            #print("ni ", len(new_index))
            #print("dec ", decisions)
            for i in range(active_games):
                if (decisions[i] is None):
                    state[new_index[i]] = False
                    active_games -= 1
                else:
                    pos_tensor[new_index[i]] = pos[decisions[i]]
                    game_length[new_index[i]] += 1.
                    last_logit[new_index[i]] = logits[decisions[i]]
            if (active_games == 0):
                break
           
        return game_length, last_logit

In [5]:
tensor_dir = "data/tensors/"
x_dict = torch.load(tensor_dir + "x.pth")
x = x_dict['x'][3056:3060]
#x = x.unsqueeze(0)
x = x.repeat_interleave(10, dim=0)
print(x.shape)

torch.Size([40, 9, 9])


In [6]:
from blockulib.data import DataTransformer

class DeepSearch(ModelBasedLoop):
    def __init__(self):
        self.generator = blockulib.BlockGenerator()
        self.probe = Probe()
        self.dt = DataTransformer()
    
    def get_model_pred(self, data, batch_size, probes_per_pos = 5):
        if (data.shape[0] == 0):
            return torch.tensor([])
        gls, lls = [], []
        for i in range(0, data.shape[0], batch_size):
            batch = data[i:i+batch_size]
            batch = batch.repeat_interleave(probes_per_pos, dim = 0)
            gl, ll = self.probe(pos_tensor = batch, temperature = 0.7, top_k = 5)
            gls.append(gl)
            lls.append(ll)
        GLs = torch.cat(gls)
        LLs = torch.cat(lls)
        GLs += self.dt.logits_to_nums(LLs)
        LLs = self.dt.nums_to_logits(GLs)
        LLs = LLs.view(data.shape[0], probes_per_pos).mean(dim = 1)
        return LLs
                

In [7]:
DS = DeepSearch()

In [8]:
"""
import time
start_time = time.time()

data = DS(num_games = 10, batch_size = 200, temperature = 0.6, top_k = 4)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds") """

'\nimport time\nstart_time = time.time()\n\ndata = DS(num_games = 10, batch_size = 200, temperature = 0.6, top_k = 4)\n\nend_time = time.time()\nelapsed_time = end_time - start_time\nprint(f"Elapsed time: {elapsed_time:.6f} seconds") '

In [23]:
from blockulib.data import DataOrganizer

In [30]:
dorg = DataOrganizer()
dorg(iteration = 0)

torch.Size([8196])


In [9]:
""" for i in range(10):
    print(len(data[i])) """

' for i in range(10):\n    print(len(data[i])) '

In [10]:
import time
from blockulib.playing import play_games

start_time = time.time()

play_games(num_games=400, batch_size=10, playing_loop = DeepSearch, save = True, temperature = 0.6, top_k = 4)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Reminder: Playing loop batch size set to 200
Mean MPG : 19.49
Elapsed time: 691.421175 seconds
